<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Quantitative Finance

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

# Machine Learning

With **`TensorFlow/Keras`**.

In [ ]:
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
%config InlineBackend.figure_format = 'svg'

## Black-Scholes-Merton

Based on **supervised learning** (features and labels data). 

The idea is to learn the BSM (1973) option prices from parameter and value data.

### The Data

In [ ]:
from itertools import product

In [ ]:
# on Google Colab execute:
#!wget https://raw.githubusercontent.com/yhilpisch/py4qf/main/bsm.py

In [ ]:
from bsm import bsm_call_value

In [ ]:
bsm_call_value(S0=100, K=105, T=1, r=0.05, sigma=0.2)

In [ ]:
N = 5

In [ ]:
S0_ = np.linspace(80, 120, N)
S0_

In [ ]:
K_ = np.linspace(80, 120, N)
K_

In [ ]:
T_ = np.linspace(0.5, 1.5, N)
T_

In [ ]:
r_ = np.linspace(0.0, 0.05, N)
r_

In [ ]:
sigma_ = np.linspace(0.1, 0.2, N)
sigma_

In [ ]:
data = pd.DataFrame()

In [ ]:
%%time
for S0, K, T, r, sigma in product(S0_, K_, T_, r_, sigma_):
    value = bsm_call_value(S0, K, T, r, sigma)
    res = pd.DataFrame({'S0': S0, 'K': K, 'T': T, 'r': r,
                        'sigma': sigma, 'value': value}, index=[0])
    data = pd.concat((data, res), ignore_index=True)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
f = list(data.columns[:5])
f

In [ ]:
mu, std = data.mean(), data.std()

In [ ]:
data_ = (data - mu) / std 

### TensorFlow

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import mean_squared_error

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=len(f)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')

In [ ]:
%%time
model.fit(data_[f], data['value'],
          epochs=1000, verbose=False)

In [ ]:
data['estimate'] = model.predict(data_[f])

In [ ]:
data.head()

In [ ]:
mean_squared_error(data['value'], data['estimate'])

In [ ]:
data[['value', 'estimate']].plot();

## Pricing with the DNN

In [ ]:
bsm_call_value(102.5, 107.5, 0.8, 0.015, 0.175)

In [ ]:
bsm_call_value(102.5, 107.5, 0.8, 0.0175, 0.15)

In [ ]:
d = pd.DataFrame(np.array((
    (102.5, 107.5, 0.8, 0.015, 0.175),
    (102.5, 107.5, 0.8, 0.0175, 0.15))), columns=f)
d

In [ ]:
d_ = (d - mu[f]) / std[f]

In [ ]:
d_

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
model.predict(d_)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>